WIP notebook to build bold diffing workflow

In [ ]:
from neurovolume.functions import *

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

Let's do something as canonical as possible:
- Assume that experimental and control are properly timed (you can add offsets later)

In [ ]:
import nibabel as nib
experimental = nib.load('/Users/joachimpfefferkorn/repos/neurovolume/media/openneuro/sub-01_task-emotionalfaces_run-1_bold.nii').get_fdata()
control = nib.load('/Users/joachimpfefferkorn/repos/neurovolume/media/openneuro/sub-01_task-rest_bold.nii').get_fdata()

- [ ] **Add an offset for the control!**

In [ ]:
method_of_subtraction = create_normalized_volume(np.clip(np.subtract(experimental, control[:,:,:,:experimental.shape[3]]), 0, None))
print(method_of_subtraction.shape)

In [ ]:
fig, axs = plt.subplots(1, 3)
z, t, = 16, 92

axs[0].imshow(experimental[:,:,z,t])
axs[1].imshow(control[:,:,z,t])
axs[2].imshow(method_of_subtraction[:,:,z,t])

We could easily export this, but let's:
- extract *just* the motion so we can see what we're looking at
- add some time correction and frame interpolation! 

In [ ]:
motion_extraction = create_normalized_volume(np.clip(np.diff(method_of_subtraction, axis=3), 0, None))

Notice that we lose a frame here!

In [ ]:
print(motion_extraction.shape)
print(method_of_subtraction.shape)

In [ ]:
# explore_4D_vol(motion_extraction)
# print(motion_extraction.min(), motion_extraction.max())
# plot_average(motion_extraction)

Great, let's stretch this out into realtime!

In [ ]:
exp_hdr = nib.load('/Users/joachimpfefferkorn/repos/neurovolume/media/openneuro/sub-01_task-emotionalfaces_run-1_bold.nii').header
seconds_per_frame = exp_hdr['pixdim'][4] #seconds per frame
fps = 24
frame_duration = seconds_per_frame * fps
total_frames = int(frame_duration * motion_extraction.shape[3])

print(f'frames last {seconds_per_frame} seconds')
print(f'Project is {fps} frames per second')
print(f'frames last {frame_duration} frames (or at least that is the interval between each non-interpolated frame)')
print(f'Project total frames: {total_frames}')

In [ ]:
stretched = np.empty((motion_extraction.shape[0], motion_extraction.shape[1], motion_extraction.shape[2], total_frames))
#Not super scalable if you have a non-int seconds per frame
print(stretched.shape)

cross dissolve from frame to frame:

In [ ]:
if_idx = 0 # how many steps you are from the last og frame
og_frame = 0
a_frame = np.empty_like(motion_extraction[:,:,:,0])
b_frame = np.empty_like(motion_extraction[:,:,:,0])

for frame in range(stretched.shape[3]):
    if float(frame % frame_duration) == 0.0: #og frame
        if_idx = 0
        a_frame = motion_extraction[:,:,:,int(frame/frame_duration)]
        stretched[:,:,:, frame] = a_frame
        if og_frame + 1 <= motion_extraction.shape[3]-1:
            b_frame = motion_extraction[:,:,:,int(og_frame+1)]
        og_frame += 1

    else: #interframe
        if_idx += 1
        a_scalar = (frame_duration - if_idx) / frame_duration
        b_scalar = if_idx / frame_duration

        stretched[:,:,:,frame] = ((a_frame * a_scalar) + (b_frame * b_scalar)) #maybe there's a faster numpy function?

In [ ]:
print(stretched.min(), stretched.max())

In [ ]:
#typically takes seven seconds
plot_average(stretched)

In [ ]:
explore_4D_vol(stretched[:,:,:,0:96], dim='z', cmap='viridis')

In [ ]:
output = '/Users/joachimpfefferkorn/repos/neurovolume/output/bold/motion_extraction_stretch.npy'
with open(output, 'wb') as f:
    np.save(f, stretched)